In [4]:
import wbdata
import pandas as pd
import pandas_profiling
import datetime
import quandl
%config IPCompleter.greedy=True
import pandas_datareader
import urllib3

ImportError: cannot import name 'StringIO' from 'pandas.compat' (C:\Users\leomf\Anaconda3\lib\site-packages\pandas\compat\__init__.py)

In [ ]:
!pip uninstall pandas_datareader

In [ ]:
### fecthing country names from IMF database to get all ISO codes
countrycodes = pd.read_csv('coucodes.csv', delimiter = ';', encoding = "ISO-8859-1")
Eastern = countrycodes["Region (IMF)"] == "Eastern Europe"
Western = countrycodes["Region (IMF)"] == "Western Europe"
countrycodes = countrycodes[Eastern | Western]
countrycodes = countrycodes[["IMF Name", "ISO Code", "ISO2 Code"]]
countrycodes.reset_index(level=0, inplace=True)
countrycodes = countrycodes.rename(columns={'IMF Name': 'country'})

ECB_countries = ["Austria", "Belgium", "Denmark", "Finland", "France", "Germany", "Greece", "Ireland","Italy", "Luxembourg", "Netherlands", "Portugal", "Spain", "Sweden", "United Kingdom"]
countrycodes = countrycodes[countrycodes["country"].isin(ECB_countries)]

In [ ]:
# Example: loading IMF data into pandas
#Monthly data
# Import libraries
import requests
import pandas as pd

url = "http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q."

#Countries
couvalues = list(countrycodes["ISO2 Code"].values)
iso2 = countrycodes["ISO2 Code"].str.cat(sep='+')
iso3 = countrycodes["ISO Code"].str.cat(sep=',')

#Variables
variables = ".PCPI_IX+FIGB_PA+BCAXF_BP6_USD+NGDP_R_K_IX+LWR_IX+AIP_SA_IX+FPE_IX.?startPeriod=1976&endPeriod=2018"

url = url+iso2+variables

# Get data from the above URL using the requests package
data = requests.get(url).json()


#### stacking IMF data and creating a dataframe
stack = []
data2 = pd.DataFrame()
for x in range(len(data['CompactData']['DataSet']['Series'])-1):
    data2 = pd.DataFrame(data['CompactData']['DataSet']['Series'][x]["Obs"])[["@OBS_VALUE", "@TIME_PERIOD"]]
    data2["country"] = data['CompactData']['DataSet']['Series'][x]["@REF_AREA"]
    data2["indicator"] = data['CompactData']['DataSet']['Series'][x]["@INDICATOR"]
    stack.append(data2)
stack = pd.concat(stack)


stack = stack.set_index(["@TIME_PERIOD", 'country', 'indicator']).unstack(level=-1)
stack.columns = stack.columns.droplevel(0)

stack.reset_index(inplace=True)


stack = stack.rename(columns={'@TIME_PERIOD': 'year', "country": "ISO2 Code"})
#Next: Analyze the missing data and data range

import missingno as msno
msno.matrix(stack)
stack = pd.merge(stack, countrycodes[["ISO2 Code", "ISO Code"]], on = "ISO2 Code")

In [ ]:
from cif import cif

data_all, subjects_all, measures_all = cif.createDataFrameFromOECD(countries = countrycodes["ISO Code"].tolist(), frequency = 'Q', subject = ["SPASTT01"])


In [ ]:
oecd = data_all.stack(level = 0)
oecd.columns = oecd.columns.droplevel(0)
oecd = oecd.reset_index()
oecd = oecd.drop(['GP', 'GY'], axis=1)  


In [ ]:
full = pd.merge(stack, oecd, left_on = ["ISO Code", "year"], right_on = ["country", "level_0"], how = "left")



In [ ]:
full = full.drop(["AIP_SA_IX", "LWR_IX", "FPE_IX", "ISO Code", "level_0"], axis = 1)
msno.matrix(full)

In [ ]:
from io import BytesIO
import pandas as pd
from urllib.request import urlopen
from zipfile import ZipFile

z = urlopen('https://www.bis.org/statistics/full_bis_total_credit_csv.zip')
myzip = ZipFile(BytesIO(z.read())).extract('WEBSTATS_TOTAL_CREDIT_DATAFLOW_csv_col.csv')
credit =pd.read_csv(myzip)


credit = credit[(credit["Borrowing sector"] == "Private non-financial sector") | (credit["Borrowing sector"] == "General government")]
credit = credit[(credit["Lending sector"] == "All sectors")]
credit = credit[(credit["BORROWERS_CTY"].isin(countrycodes["ISO2 Code"]))]
credit = credit[(credit["Unit type"] == "Percentage of GDP")]
credit = credit[(credit["TC_ADJUST"] == "A")]
credit = credit[(credit["Valuation"] == "Market value")]
credit = (credit.set_index(['Borrowing sector', 'BORROWERS_CTY'])
   .rename_axis(['Year'], axis=1)
   .stack()
   .unstack('Borrowing sector')
   .reset_index())
credit = credit.iloc[13:]

In [ ]:
z = urlopen('https://www.bis.org/statistics/full_bis_dsr_csv.zip')
myzip = ZipFile(BytesIO(z.read())).extract('WEBSTATS_DSR_DATAFLOW_csv_col.csv')
debt_service =pd.read_csv(myzip)

debt_service = debt_service[(debt_service["Borrowers"] == "Private non-financial sector")]
debt_service = debt_service[(debt_service["BORROWERS_CTY"].isin(countrycodes["ISO2 Code"]))]
debt_service = (debt_service.set_index(['BORROWERS_CTY', "Borrowers"])
   .rename_axis(['Year'], axis=1)
   .stack()
   .unstack("Borrowers")
   .reset_index())

debt_service = debt_service.iloc[5:]
debt_service.columns = debt_service.columns = ["BORROWERS_CTY", "Year", "debt_service"]

In [ ]:
house_prices = pd.read_csv("https://stats.oecd.org/sdmx-json/data/DP_LIVE/.HOUSECOST.../OECD?contentType=csv&detail=code&separator=comma&csv-lang=en")
house_prices = house_prices[(house_prices["FREQUENCY"] == "Q")]
house_prices = house_prices[(house_prices["LOCATION"].isin(countrycodes["ISO Code"]))]
house_prices = house_prices[(house_prices["SUBJECT"] == "PRICEINCOME")]
house_prices = house_prices.drop(columns=["INDICATOR", "SUBJECT", "MEASURE", "FREQUENCY", "Flag Codes"])
house_prices.columns = house_prices.columns = ["LOCATION", "TIME", "House_prices"]

In [ ]:
full = pd.merge(full, credit, left_on = ["ISO2 Code", "year"], right_on = ["BORROWERS_CTY", "Year"], how = "left")
full = pd.merge(full, debt_service, left_on = ["ISO2 Code", "year"], right_on = ["BORROWERS_CTY", "Year"], how = "left")
full = pd.merge(full, house_prices, left_on = ["country", "year"], right_on = ["LOCATION", "TIME"], how = "left")

In [ ]:
full = full.drop(columns = ["country", "Year_x", "BORROWERS_CTY_x", "BORROWERS_CTY_y", "Year_y", "LOCATION", "TIME"])
full[["BCAXF_BP6_USD", "FIGB_PA", "NGDP_R_K_IX", "PCPI_IX", "IXOB", "General government", "Private non-financial sector", "debt_service", "House_prices"]] = full[["BCAXF_BP6_USD", "FIGB_PA", "NGDP_R_K_IX", "PCPI_IX", "IXOB", "General government", "Private non-financial sector", "debt_service", "House_prices"]].apply(pd.to_numeric)


In [ ]:
full["date"] = pd.to_datetime(full['year'])
full1990 = full[full["date"] > "1990-01-01"]

In [ ]:
full["gdp_growth"] = full.groupby('ISO2 Code', sort=False).NGDP_R_K_IX.apply(
     lambda x: x.pct_change(4))

full.rename(columns={'Private non-financial sector':'credit_private', 'General government': 'credit_government'}, inplace=True)

full["credit_growth"] = full.groupby('ISO2 Code', sort=False).credit_private.apply(
     lambda x: x.pct_change(4))

full["inflation"] = full.groupby('ISO2 Code', sort=False).PCPI_IX.apply(
     lambda x: x.pct_change(4))

full["stock_growth"]  = full.groupby('ISO2 Code', sort=False).IXOB.apply(
     lambda x: x.pct_change(4))

full["house_growth"]  = full.groupby('ISO2 Code', sort=False).House_prices.apply(
     lambda x: x.pct_change(4))


In [ ]:
import statsmodels.api as sm

### Creating credit gap variable

groups = full.groupby('ISO2 Code')

group_keys = list(groups.groups.keys())


bs = pd.DataFrame()

for key in group_keys:

    g = groups.get_group(key).copy()
    target = g['credit_private']

    cycle, trend = sm.tsa.filters.hpfilter(target, lamb=129600)

    g['credit_gap'] = trend
    bs = bs.append(g)

bs

In [ ]:
### Creating house price gap variable
groups = bs.groupby('ISO2 Code')

group_keys = list(groups.groups.keys())


bs2 = pd.DataFrame()

for key in group_keys:

    g = groups.get_group(key).copy()
    target = g['House_prices']

    cycle, trend = sm.tsa.filters.hpfilter(target, lamb=400000)

    g['house_gap'] = trend
    bs2 = bs2.append(g)

bs2


In [ ]:
### fetching crisis database from ecb paper
ecb = pd.read_csv('data/ecbdata.csv', error_bad_lines = False, delimiter = ';')
                  

In [ ]:
ecb.head()

In [ ]:
ecb = ecb[ecb['Country'].isin(full['ISO2 Code'])]

In [ ]:
### Crisis database taken from the ecb paper
ecb


In [ ]:
ecb.rename(columns={'Start date':'start'}, inplace=True)
ecb.head()

In [ ]:
### turning to quarters

ecb['year'] = pd.PeriodIndex(pd.to_datetime(ecb.start), freq = 'Q').astype(str)

In [ ]:
crisis = ecb[['Country', 'year']]
crisis = crisis.copy()


In [ ]:
bs2["year"] = bs2["year"].str.replace("-", "")


In [ ]:
crisis["dummy"] = 1

In [ ]:
### Creating crises variable
crise = pd.merge(bs2, crisis, left_on = ['ISO2 Code', 'year'], right_on = ['Country', 'year'], how = 'left')

In [ ]:
crisis.dtypes

In [ ]:
crise['dummy'].fillna(0, inplace = True)

In [ ]:
### Merged database with quarter of crisis information
crise['dummy'].describe()

In [ ]:
#Creating the lags of dependent variable
#Backwards
crise["lag_1"] =crise.groupby("ISO2 Code")["dummy"].shift(-1)
crise["lag_2"] =crise.groupby("ISO2 Code")["dummy"].shift(-2)
crise["lag_3"] =crise.groupby("ISO2 Code")["dummy"].shift(-3)
crise["lag_4"] =crise.groupby("ISO2 Code")["dummy"].shift(-4)
crise["lag_5"] =crise.groupby("ISO2 Code")["dummy"].shift(-5)
crise["lag_6"] =crise.groupby("ISO2 Code")["dummy"].shift(-6)
crise["lag_7"] =crise.groupby("ISO2 Code")["dummy"].shift(-7)
crise["lag_8"] =crise.groupby("ISO2 Code")["dummy"].shift(-8)
crise["lag_9"] =crise.groupby("ISO2 Code")["dummy"].shift(-9)
crise["lag_10"] =crise.groupby("ISO2 Code")["dummy"].shift(-10)
crise["lag_11"] =crise.groupby("ISO2 Code")["dummy"].shift(-11)
crise["lag_12"] =crise.groupby("ISO2 Code")["dummy"].shift(-12)

#Front
crise["lag_1f"] =crise.groupby("ISO2 Code")["dummy"].shift(1)
crise["lag_2f"] =crise.groupby("ISO2 Code")["dummy"].shift(2)
crise["lag_3f"] =crise.groupby("ISO2 Code")["dummy"].shift(3)
crise["lag_4f"] =crise.groupby("ISO2 Code")["dummy"].shift(4)
crise["lag_5f"] =crise.groupby("ISO2 Code")["dummy"].shift(5)
crise["lag_6f"] =crise.groupby("ISO2 Code")["dummy"].shift(6)
crise["lag_7f"] =crise.groupby("ISO2 Code")["dummy"].shift(7)
crise["lag_8f"] =crise.groupby("ISO2 Code")["dummy"].shift(8)



crise["one_year"] = crise["lag_1"] + crise["lag_2"] + crise["lag_3"] +crise["lag_4"]
crise["two_year"] = crise["one_year"] +crise["lag_5"] + crise["lag_6"] + crise["lag_7"] +crise["lag_8"]
crise["three_year"] = crise["two_year"] + crise["lag_9"] + crise["lag_10"] + crise["lag_11"] +crise["lag_12"]

crise["one_year_f"] = crise["lag_1f"] + crise["lag_2f"] + crise["lag_3f"] +crise["lag_4f"]
crise["two_year_f"] = crise["one_year_f"] +crise["lag_5f"] + crise["lag_6f"] + crise["lag_7f"] +crise["lag_8f"]


In [ ]:
#Removing 

#crise = crise[crise['dummy'] == 0]
#crise = crise[crise["two_year_f"] == 0]
#crise = crise[crise["one_year"] == 0]


In [ ]:
crise.columns

In [ ]:
crise.isna().sum()

In [ ]:
### Preparing machine learning pipeline 

from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier


In [ ]:
crise = crise[crise["date"] < "2018-01-01"]
X =  crise[['ISO2 Code','BCAXF_BP6_USD', 'FIGB_PA', 'credit_government', 'credit_private', 'debt_service', 'House_prices', 'gdp_growth', 'credit_growth', 'inflation', 'stock_growth', 'credit_gap', 'house_gap', "house_growth"]]
Y = crise['three_year']


In [ ]:
### filling nas in database.

X.groupby("ISO2 Code").ffill()
Y.fillna(0, inplace = True)
X.fillna(X.median(), inplace = True)

X = pd.get_dummies(X, columns=['ISO2 Code'], prefix = ['Country'])

In [ ]:
### train test split

X_train, X_test, y_train, y_test =train_test_split(X,Y,
                                                   test_size=0.25,
                                                   random_state=0,
                                                   stratify=crise['three_year'])

In [ ]:
# Spot-Check Algorithms
def GetBasedModel():
    basedModels = []
    basedModels.append(('LR'   , LogisticRegression()))
    basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
    basedModels.append(('KNN'  , KNeighborsClassifier()))
    basedModels.append(('CART' , DecisionTreeClassifier()))
    basedModels.append(('NB'   , GaussianNB()))
    basedModels.append(('SVM'  , SVC(probability=True)))
    basedModels.append(('AB'   , AdaBoostClassifier()))
    basedModels.append(('GBM'  , GradientBoostingClassifier()))
    basedModels.append(('RF'   , RandomForestClassifier()))
    basedModels.append(('ET'   , ExtraTreesClassifier()))

    
    return basedModels

In [ ]:
from sklearn.metrics import confusion_matrix


def BasedLine2(X_train, y_train,models):
    # Test options and evaluation metric
    num_folds = 10
    scoring = 'roc_auc'

    results = []
    names = []
    for name, model in models:
        kfold = StratifiedKFold(n_splits=num_folds, random_state= 1990)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    return names, results

In [ ]:
from plotly import graph_objs as go
import sys
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

class PlotBoxR(object):
    
    
    def __Trace(self,nameOfFeature,value): 
    
        trace = go.Box(
            y=value,
            name = nameOfFeature,
            marker = dict(
                color = 'rgb(0, 128, 128)',
            )
        )
        return trace

    def PlotResult(self,names,results):
        
        data = []

        for i in range(len(names)):
            data.append(self.__Trace(names[i],results[i]))


        py.iplot(data)

In [ ]:
from plotly import graph_objs as go

In [ ]:
models = GetBasedModel()
names,results = BasedLine2(X_train, y_train,models)
PlotBoxR().PlotResult(names,results)

In [ ]:
from sklearn import preprocessing

X_2 = preprocessing.scale(X)



X_train, X_test, y_train, y_test =train_test_split(X_2,Y,
                                                   test_size=0.3,
                                                   random_state=0,
                                                   stratify=Y)

model = ExtraTreesClassifier(n_jobs=2, min_samples_split = 2)

gsc = GridSearchCV(
    estimator=model,
    param_grid={
        'n_estimators': range(1600, 1601, 1),
        
    },
    scoring='roc_auc',
    cv=10
)

grid_result = gsc.fit(X_train, y_train)


In [ ]:
grid_result.best_params_

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

for test_mean, train_mean, param in zip(
        grid_result.cv_results_['mean_test_score'],
        grid_result.cv_results_['mean_train_score'],
        grid_result.cv_results_['params']):
    print("Train: %f // Test : %f with: %r" % (train_mean, test_mean, param))
    
model = ExtraTreesClassifier(**grid_result.best_params_)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)


In [ ]:


a = model.predict_proba(X_2[crise["ISO2 Code"] == "PT"])[:,1]



pt = crise[crise["ISO2 Code"] == "PT"]
pt["a"] = a
pt = pt.set_index('year')
pt = pd.merge(crise, pt, on = ['ISO2 Code', 'year'], how = "left")
pt = pt[pt["ISO2 Code"] == "PT"]
pt["year"] = pd.to_datetime(pt['year'])
pt = pt.set_index('year')
ax = pt['a'].plot(linewidth=0.5)
ax.axvline("2010-01-01")


In [ ]:
import seaborn as sns

a = model.predict_proba(X_2)[:,1]



pt = crise
pt["a"] = a
pt = pt.set_index('year')
pt = pd.merge(crise, pt, on = ['ISO2 Code', 'year'], how = "left")
pt["year"] = pd.to_datetime(pt['year'])
pt = pt.set_index('year')
g = sns.FacetGrid(pt, col="ISO2 Code")
g = g.map(plt.plot, "year", "a")

In [ ]:
def ScoreDataFrame(names,results):
    def floatingDecimals(f_val, dec=3):
        prc = "{:."+str(dec)+"f}" 
    
        return float(prc.format(f_val))

    scores = []
    for r in results:
        scores.append(floatingDecimals(r.mean(),4))

    scoreDataFrame = pd.DataFrame({'Model':names, 'Score': scores})
    return scoreDataFrame

In [ ]:
basedLineScore = ScoreDataFrame(names,results)
basedLineScore

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


def GetScaledModel(nameOfScaler):
    
    if nameOfScaler == 'standard':
        scaler = StandardScaler()
    elif nameOfScaler =='minmax':
        scaler = MinMaxScaler()

    pipelines = []
    pipelines.append((nameOfScaler+'LR'  , Pipeline([('Scaler', scaler),('LR'  , LogisticRegression())])))
    pipelines.append((nameOfScaler+'LDA' , Pipeline([('Scaler', scaler),('LDA' , LinearDiscriminantAnalysis())])))
    pipelines.append((nameOfScaler+'KNN' , Pipeline([('Scaler', scaler),('KNN' , KNeighborsClassifier())])))
    pipelines.append((nameOfScaler+'CART', Pipeline([('Scaler', scaler),('CART', DecisionTreeClassifier())])))
    pipelines.append((nameOfScaler+'NB'  , Pipeline([('Scaler', scaler),('NB'  , GaussianNB())])))
    pipelines.append((nameOfScaler+'SVM' , Pipeline([('Scaler', scaler),('SVM' , SVC())])))
    pipelines.append((nameOfScaler+'AB'  , Pipeline([('Scaler', scaler),('AB'  , AdaBoostClassifier())])  ))
    pipelines.append((nameOfScaler+'GBM' , Pipeline([('Scaler', scaler),('GMB' , GradientBoostingClassifier())])  ))
    pipelines.append((nameOfScaler+'RF'  , Pipeline([('Scaler', scaler),('RF'  , RandomForestClassifier())])  ))
    pipelines.append((nameOfScaler+'ET'  , Pipeline([('Scaler', scaler),('ET'  , ExtraTreesClassifier())])  ))


    return pipelines 


In [ ]:
models = GetScaledModel('standard')
names,results = BasedLine2(X_train, y_train,models)
PlotBoxR().PlotResult(names,results)
scaledScoreStandard = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard], axis=1)
compareModels

In [ ]:
model = ExtraTreesClassifier(n_estimators=250,
                              random_state=1990)

model.fit(X_train, y_train)




In [ ]:
models = GetScaledModel('minmax')
names,results = BasedLine2(X_train, y_train,models)
PlotBoxR().PlotResult(names,results)

scaledScoreMinMax = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard,
                          scaledScoreMinMax], axis=1)
compareModels

In [ ]:
### feature importance

clf = ExtraTreesClassifier(n_estimators=250,
                              random_state=1990)

clf.fit(X_train, y_train)

# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X.columns[sorted_idx])#boston.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
model = clf.fit(X_train, y_train)

In [ ]:
### Model interpretability via shapley values

import shap

shap_values = shap.TreeExplainer(model).shap_values(X_train)


In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
# setting show=False allows us to continue customizing the matplotlib plot before displaying it
shap.dependence_plot("inflation", shap_values, X_train, show=False)
pl.xlim(80,225)
pl.show()